In [53]:
import typing
import datetime
import requests

import dotenv
import pydantic
import rich

In [54]:
import cltrier_lib as lib

In [55]:
ENV = dotenv.dotenv_values()

In [56]:
REQUEST_ENDPOINT: str = ENV["OLLAMA_ENDPOINT"]
REQUEST_META: typing.Dict = {
    "headers": {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ENV["OLLAMA_BEARER"]}"
    }
}

In [57]:
class BaseResponse(pydantic.BaseModel):
    model: str

    created_at: datetime.datetime
    total_duration: datetime.timedelta

    prompt_eval_count: int
    eval_count: int

    @pydantic.field_validator("total_duration", mode="before")
    def convert_to_milliseconds(cls, val: int):
        return val * 1e-9

In [58]:
class GenerateResponse(BaseResponse):
    response: str

In [59]:
class ChatResponse(BaseResponse):
    message: lib.inference.schemas.Message

In [60]:
rich.print(GenerateResponse(**requests.post(
    f"{REQUEST_ENDPOINT}/generate/",
    json={
        "model": "llama3.1:8b-instruct-q6_K",
        "prompt": "Why is the sky blue?",
        "stream": False
    },
    **REQUEST_META
).json()))

GenerateResponse(
    model='llama3.1:8b-instruct-q6_K',
    created_at=datetime.datetime(2024, 9, 10, 8, 13, 56, 35771, tzinfo=TzInfo(UTC)),
    total_duration=datetime.timedelta(seconds=10, microseconds=133458),
    prompt_eval_count=17,
    eval_count=524,
    response="The sky appears blue to us during the day because of a phenomenon called Rayleigh scattering. Here's 
what happens:\n\n1. **Sunlight**: When sunlight enters Earth's atmosphere, it consists of all the colors of the 
visible spectrum (red, orange, yellow, green, blue, indigo, and violet). These colors are like different types of 
rays of light.\n\n2. **Scattering**: The smaller molecules in the atmosphere scatter these sun rays in all 
directions. However, they scatter shorter wavelengths (like blue) more than longer wavelengths (like red), a 
phenomenon known as Rayleigh scattering.\n\n3. **What we see**: When we look up at the sky during the day, what 
we're seeing is not just the direct sunlight from the sun itself but also all the scattered light rays from the 
entire hemisphere of the sky above us. Because blue light is scattered more than other colors and because our 
atmosphere is much more prevalent in front of us, it reaches our eyes in greater abundance, making the sky appear 
blue.\n\n4. **Other factors**: However, the color of the sky can change under different conditions:\n   - 
**Sunset**: During sunset, the sun's light has to travel through more of the Earth's atmosphere to reach us. This 
means more scattering occurs (especially for shorter wavelengths), and what we see is more red than blue because 
less blue reaches our eyes due to the increased scattering.\n   - **Clouds and Dust**: If there are thick clouds or
dust in the air, they scatter light differently compared to the Earth's atmosphere, which can change the apparent 
color of the sky. For example, during a sandstorm, the sky might appear orange or red due to the large amount of 
dust particles scattering longer wavelengths.\n   - **Atmospheric Pollution**: In some cases, pollutants in the air
like particulate matter, gases, and aerosols can scatter light differently as well, potentially altering the 
apparent color of the sky.\n\n5. **Moonlight Sky**: At night, when it's dark enough for stars to be visible, the 
moon itself reflects sunlight back onto our atmosphere (and us), but this is scattered in a way that doesn't show 
its original color due to the Earth's atmospheric conditions. This makes the night sky appear black with twinkling 
lights of the distant stars and planets.\n\nIn summary, the blue color we see during the day is primarily due to 
the scattering of sunlight by the small molecules of gases in our atmosphere, which scatters shorter wavelengths 
(like blue) more than longer ones (like red), leading to the beautiful blue hue of a clear sky."
)

In [61]:
rich.print(ChatResponse(**requests.post(
    f"{REQUEST_ENDPOINT}/chat/",
    json={
        "model": "llama3.1:8b-instruct-q6_K",
        **lib.inference.schemas.Chat(messages=[
            lib.inference.schemas.Message(
                role="user",
                content="why is the sky blue?"
            )
        ]).model_dump(),
        "stream": False
    },
    **REQUEST_META
).json()))

ChatResponse(
    model='llama3.1:8b-instruct-q6_K',
    created_at=datetime.datetime(2024, 9, 10, 8, 14, 1, 123509, tzinfo=TzInfo(UTC)),
    total_duration=datetime.timedelta(seconds=4, microseconds=973918),
    prompt_eval_count=17,
    eval_count=392,
    message=Message(
        role='assistant',
        content="The sky appears blue to us during the daytime because of a phenomenon called scattering. When 
sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These 
molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) 
wavelengths.\n\nHere's a step-by-step explanation:\n\n1. **Sunlight Enters Atmosphere**: Sunlight contains a wide 
range of electromagnetic radiation, including visible light with different wavelengths (colors).\n\n2. **Molecules 
Scattering Light**: When sunlight encounters the tiny molecules in Earth's atmosphere (mainly nitrogen and oxygen),
it causes the light to scatter in all directions.\n\n3. **Scattering Preference for Blue Wavelengths**: The 
scattering effect preferentially scatters shorter (blue) wavelengths over longer (red, orange, yellow) wavelengths.
This means that out of all the colors of sunlight, blue is scattered more than any other color.\n\n4. **What We See
from Earth's Surface**: As a result of this scattering, our line of sight to the sun during the daytime includes 
much more blue light because it has been scattered in all directions by the atmosphere. This is why we see the sky 
as blue rather than yellow or white.\n\n5. **Intensity and Color Variation with Sun’s Position**: The apparent 
color of the sky can also change depending on several factors, such as the time of day, weather conditions (like 
pollution or dust particles), and atmospheric phenomena like clouds. For example, during sunrise and sunset, when 
light travels longer distances through the atmosphere to reach our eyes, we tend to see more red hues in the sky 
due to Rayleigh scattering, which scatters shorter wavelengths more than longer ones.\n\nThis phenomenon is known 
as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in 1871. It's a 
fundamental aspect of how light interacts with matter and why our atmosphere appears blue during the day."
    )
)